# LSTMでコロナ感染者数予測

# import

In [ ]:
import torch
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython.display import Image
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler#レンジに幅がある時に標準化する
from sklearn.model_selection import train_test_split#トレーニングとテストデータを分けてかつシャッフルする
import pandas as pd
import torch.nn.functional as F#さまざまな活性化関数
from torch.utils.data import Dataset, DataLoader#IrisDatasetでDatasetというモデルを継承しているが、これを使用出来るようにするため左記のimportが必要
from torchvision import datasets,transforms #MNIST画像はpytorchで準備されてる。
%matplotlib inline
#%matplotlib inlineを指定した時の挙動
#１）グラフがアウトプット行b（in line）に出力される。いちいちポップアップが出ない。
#２）plt.show()を省略してもグラフが出力される。対話型実行環境やエディタでは、グラフの表示のためにplt.show()を実行する必要。
#    →plt.show()を省略するとアウトプット行のグラフの上にオブジェクトの場所が出力される。

# ライブラリのImport

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pickle
from model import LSTM_Corona
#エラーが発生するのを防ぐためにLSTM_Coronaを外部のモジュールする。
#  ・エラー内容：
#  　jupyterでLSTMモデルクラスであるLSTM_Corona作成し学習後にpickle化して出力。
#  　→streamlit.pyでpickleを読み込んで予測を行おうとしたら下記のエラーになった。
#     「AttributeError ：Can't get attribute LSTM_Corona on <module  '__main__'  from 'streamlit.py'>」
#       　→__main__モジュールにLSTM_Cornaがないと言われている。
#         →jupyter上で__main__.LSTM_Cornaというクラスをpickle化したため＝jupyter上でLSTM_Cornaというクラスを作成したら__main__.LSTM_Cornaモジュールになる。
#         →pythonコマンドで実行したファイルは、__main__モジュールとして扱われるため。
#  ・解決方法：
#      別途model.pyを作成しその中にLSTM_Coronaを記載する。そしてそれをjupyterファイルと同じ階層に置いて、jupyterファイル内でimportする。
#       →そしてjupyterファイル内ではLSTM_Coronaを定義しないことにより__main__.LSTM_Cornaモジュールではなく、model.LSTM_Cornaモジュールになる。
#       →その結果エラーが発生しなくなる。

# データパス

In [ ]:
covid19_data = './data/lstm/time_series_covid19_confirmed_global.csv'#RNNで使用するCOVID19データ

# RNN_COVID19

In [ ]:
from pandas.plotting import register_matplotlib_converters

In [ ]:
df = pd.read_csv(covid19_data)
df.head()
#Province/State 州
#Lat Long 緯度と軽度
#上記以降は日ベースでの感染された数

In [ ]:
#データの中で0で変化がないところを削る。またRegionなど文字情報も不要なので削る
#最初の37列を削る。
df = df.iloc[:,37:]
df.head()

In [ ]:
#行は278行（＝278カ国地域）。
df

In [ ]:
#日ベースごとに全世界を足して、各日ベースの世界全体の感染者数求める
daily_global = df.sum(axis=0)
daily_global

In [ ]:
#日付の値をpandasのdatetimeに変換
daily_global.index = pd.to_datetime(daily_global.index)
daily_global

In [ ]:
plt.plot(daily_global)

In [ ]:
#直近の30日をテストデータにして、3日より前をトレーニングデータにする。
y=daily_global.values.astype(float)

test_size = 30
train_original_data = y[:-test_size]#最後の30日分を取り除いた、それより以前をトレーニングにする。
test_original_data = y[-test_size:]

In [ ]:
#入力のデータを正規化（-1〜0に収まるように変換）
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))

In [ ]:
#scalerをデータに適合させる
#sklearnで使えるように１列になるようにデータ整える
train_normalized = scaler.fit_transform(train_original_data.reshape(-1,1))
train_normalized.shape

In [ ]:
#Tensor型に変換
train_normalized = torch.FloatTensor(train_normalized).view(-1)
#１ヶ月分を予測するようにする。
#そのためwindow_sizeを3から30に変更
window_size = 30

In [ ]:
def sequence_creator(input_data,window):
    dataset = []
    data_len = len(input_data)
    for i in range(data_len - window):
        window_fr = input_data[i:i+window]
        label = input_data[i+window:i+window+1]
        dataset.append((window_fr, label))
    return dataset

In [ ]:
print(len(train_normalized))

In [ ]:
train_data = sequence_creator(train_normalized, window_size)#ラベル付けを自動でさせる

In [ ]:
# train_data

In [ ]:
# class LSTM_Corona(nn.Module):
#     def __init__(self, in_size=1, h_size=30, out_size=1):
#         super().__init__()
#         self.h_size = h_size
#         self.lstm = nn.LSTM(in_size,h_size)
#         self.fc = nn.Linear(h_size,out_size)
        
#         self.hidden = (torch.zeros(1,1,h_size),torch.zeros(1,1,h_size))
        
#     def forward(self, sequence_data):
#         #lstmを実行するときは3次元のサイズを指定する必要がある。
#         #１つ目の引数＝1次元目＝データのサイズ（len(sequence_data)）＝今回は30個 = train_dataNの中は30
#         #2つ目の引数＝2次元目＝バッチサイズ＝今回はバッチ化していないので1
#         #3つ目の引数＝3次元目＝隠れ層のサイズ＝今回なら引数で指定した30。
#         lstm_out, self.hidden = self.lstm(sequence_data.view(len(sequence_data),1,-1),self.hidden)
#         pred=self.fc(lstm_out.view(len(sequence_data),-1))
        
#         return pred[-1]#欲しいのは最後のデータ

In [ ]:
torch.manual_seed(3)
model = LSTM_Corona()
criterion = nn.MSELoss()#損失関数=平均二乗誤差。
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)#最適化関数

In [ ]:
epochs = 100

for epoch in range(epochs):
    for sequence_in, y_train in train_data:
        
        y_pred = model(sequence_in)
        loss = criterion(y_pred, y_train)

        optimizer.zero_grad()
        model.hidden = (torch.zeros(1,1,model.h_size),torch.zeros(1,1,model.h_size))
        
        loss.backward()
        optimizer.step()
        
    print(f'Epoech {epoch+1} Loss {loss.item():.3f}')

In [ ]:
#１ヶ月分を予測するようにする。
test = 30

preds = train_normalized[-window_size:].tolist()

#評価モード
model.eval()

for i in range(test):
    sequence = torch.FloatTensor(preds[-window_size:])
    with torch.no_grad():#勾配の計算を無効化
        model.hidden = (torch.zeros(1,1,model.h_size), torch.zeros(1,1,model.h_size))#隠れ層リセット
        preds.append(model(sequence).item())

In [ ]:
#予測した値から正規化を解いてやる
predictions = scaler.inverse_transform(np.array(preds[window_size:]).reshape(-1,1))
predictions

In [ ]:
#テストの値
#１ヶ月分を予測するようにする。
daily_global[-30:]
#テストの値とpredictionsの値が比較的近い値になっている

In [ ]:
#１ヶ月分を予測するようにする。
x = np.arange('2021-11-14','2021-12-14', dtype='datetime64[D]').astype('datetime64[D]')
x

In [ ]:
plt.figure(figsize=(12,5))
plt.grid(True)
plt.plot(daily_global)#元データ表示
plt.plot(x,predictions)#最後の3日だけ予測値重ね合わせる
plt.show()

In [ ]:
#未知のデータの予測
epochs = 200
model.train()#トレーニングモード：全てのデータでトレーニングする。

y_normalized = scaler.fit_transform(y.reshape(-1,1))
y_normalized = torch.FloatTensor(y_normalized).view(-1)
full_data = sequence_creator(y_normalized, window_size)

In [ ]:
for epoch in range(epochs):
    for sequence_in, y_train in full_data:
        
        y_pred = model(sequence_in)
        loss = criterion(y_pred, y_train)
        
        optimizer.zero_grad()
        model.hidden = (torch.zeros(1,1,model.h_size),torch.zeros(1,1,model.h_size))
        
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1} Loss{loss.item():.3f}')

In [ ]:
#１ヶ月分を予測するようにする。
future = 30

preds = y_normalized[-window_size:].tolist()

model.eval()#評価モード

for i in range(future):
    sequence = torch.FloatTensor(preds[-window_size:])
    
    with torch.no_grad():#勾配の計算の無効化
        model.hidden =(torch.zeros(1,1,model.h_size),torch.zeros(1,1,model.h_size))#隠れ層NO無効化
        preds.append(model(sequence).item())#その都度計算された予測値を格納
        
#予測値が正規化されてるので元のスケールに戻す。
predictions = scaler.inverse_transform(np.array(preds).reshape(-1,1))

#１ヶ月分を予測するようにする。
#x = np.arange('2021-10-25','2021-11-24', dtype='datetime64[D]').astype('datetime64[D]')
x = np.arange('2021-11-14','2021-12-14', dtype='datetime64[D]').astype('datetime64[D]')

In [ ]:
print(predictions[window_size:].shape)

In [ ]:
print(x.shape)

In [ ]:
plt.figure(figsize=(12,5))
plt.title('The number of person affected by Corona virus globally')
plt.grid(True)
plt.plot(daily_global)#オリジナルデータ
plt.plot(x, predictions[window_size:])#予測値

In [ ]:
with open("./model/lstm/lstm.pickle", mode="wb") as f:
    pickle.dump(model, f)

In [ ]:
with open("./model/lstm/lstm.pickle", mode="rb") as f2:
    model1 = pickle.load(f2)

In [ ]:
#pickle化したものを読み込んで再度予測。
future = 30

preds = y_normalized[-window_size:].tolist()

model.eval()#評価モード

for i in range(future):
    sequence = torch.FloatTensor(preds[-window_size:])
    
    with torch.no_grad():#勾配の計算の無効化
        model.hidden =(torch.zeros(1,1,model.h_size),torch.zeros(1,1,model.h_size))#隠れ層NO無効化
        preds.append(model1(sequence).item())#その都度計算された予測値を格納
        
#予測値が正規化されてるので元のスケールに戻す。
predictions = scaler.inverse_transform(np.array(preds).reshape(-1,1))

#１ヶ月分を予測するようにする。
x = np.arange('2021-11-14','2021-12-14', dtype='datetime64[D]').astype('datetime64[D]')
#x = np.arange('2021-09-05','2021-09-25', dtype='datetime64[D]').astype('datetime64[D]')

In [ ]:
plt.figure(figsize=(12,5))
plt.title('The number of person affected by Corona virus globally')
plt.grid(True)
plt.plot(daily_global)#オリジナルデータ
plt.plot(x, predictions[window_size:])#予測値